In [1]:
!pip install schedule
!pip install requests

!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 11.0 MB/s eta 0:00:00


In [2]:
import requests
import csv
import schedule
import time
from datetime import datetime
import os

# Constants
STOCK_FILENAME = "cocacola_stock_data.csv"
LOG_FILENAME = "update_log_cocacola.txt"
API_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=KO&interval=5min&outputsize=full&apikey= 2RW2VSDAMEKSTSSV'

# Function to load existing timestamps from the stock data file
def load_existing_timestamps():
    if not os.path.exists(STOCK_FILENAME):
        return set()
    with open(STOCK_FILENAME, mode="r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        return {row[0] for row in reader}  # Collect all timestamps

# Function to fetch and update stock data
def fetch_stock_data():
    # Fetch data from the API
    r = requests.get(API_URL)
    data = r.json()

    if "Time Series (5min)" in data:
        time_series = data["Time Series (5min)"]

        # Load existing timestamps to avoid duplication
        existing_timestamps = load_existing_timestamps()

        # Prepare rows for new data
        new_rows = []
        for timestamp, values in time_series.items():
            if timestamp not in existing_timestamps:
                new_rows.append([
                    timestamp,
                    values["1. open"],
                    values["2. high"],
                    values["3. low"],
                    values["4. close"],
                    values["5. volume"]
                ])

        if new_rows:
            # Append new rows to the file
            file_exists = os.path.exists(STOCK_FILENAME)
            with open(STOCK_FILENAME, mode="a", newline="") as file:
                writer = csv.writer(file)
                if not file_exists:
                    # Write header if the file is new
                    header = ["Timestamp", "Open", "High", "Low", "Close", "Volume"]
                    writer.writerow(header)
                writer.writerows(new_rows)

            # Log the update
            with open(LOG_FILENAME, mode="a") as log_file:
                log_file.write(f"{datetime.now()}: Added {len(new_rows)} new rows to {STOCK_FILENAME}\n")
            print(f"Data successfully updated. Added {len(new_rows)} new rows.")
        else:
            print("No new data to update.")
    else:
        print("Error: Time Series data not found in the response")

# Schedule the task to run every hour
schedule.every(1).hour.do(fetch_stock_data)

# Run the function immediately for the first iteration
print("Fetching data for the first time...")
fetch_stock_data()

print("Scheduler is running. Press Ctrl+C to stop.")

# Keep the script running to execute the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)


Fetching data for the first time...
Data successfully updated. Added 3997 new rows.
Scheduler is running. Press Ctrl+C to stop.


KeyboardInterrupt: 